
# DT Classifier - Implementation

A naive `CART` implementation.

We will be exploring a dataset on CTL or click-through-rate prediction.

* The core of the algorithm is comes from deciding on what attribute to split on. In our implementation we iterate through every attribute in the data, and every value of that attribute (whether it be a categorical, or numerical), and compute the information gain via the entropy or gini criterion.
* Since this implementation is most for educational purposes, it comes along with its limitations.
	* Binary splits are considered for both categorical and numerical values. In other words, multiway splits are not used.
    * Rule based implementation is not included.
    * Assumes that the data is does not having missing values (does not impute data).
    * No form of tree pre or post pruning is implemented.
    * Input data is also assumed not to have missing values, and hence voting is not implemented as well.
    * This implementation obviously does not scale either as `pd.DataFrame` is used.
    * All splits are binary and not multi-way. However, it is capable of multi-way classification via one-vs-rest method.
	* Gain ratio is not considered.


In [5]:
import numpy as np


def gini_impurity(y):
    """
    objective: compute the gini impurity of a dataset
    :param y: [int] - the numerical classification or output
    :return: float
    """
    # when the set is empty, it is also pure
    if not y:
        return 0
    # count the occurrences of each label
    counts = np.unique(y, return_counts=True)[1]
    fractions = counts / float(len(y))
    return 1 - np.sum(fractions ** 2)


print(gini_impurity([1, 1, 0, 1, 0]))
print(gini_impurity([1, 1, 0, 1, 0, 0]))
print(gini_impurity([1, 1, 1, 1]))

0.48
0.5
0.0


In [6]:
import pandas as pd


# demonstrating gini impurity with a sample data set
sample_df = pd.DataFrame(data = [['m', 1, 1, 1],
                                 ['f', 0, 0, 2],
                                 ['f', 1, 1, 2],
                                 ['m', 0, 0, 1],
                                 ['m', 0, 1, 1]], 
                         columns=['gender', 'interest in tech', 'click', 'group by gender'])
sample_df

,gender,interest in tech,click,group by gender
0,m,1,1,1
1,f,0,0,2
2,f,1,1,2
3,m,0,0,1
4,m,0,1,1


In [7]:
# splitting on gender we obtain the following
sample_gender = sample_df.groupby('gender')
gender_groups = [group for key, group in sample_gender]
gender_groups

[  gender  interest in tech  click  group by gender
 1      f                 0      0                2
 2      f                 1      1                2,
   gender  interest in tech  click  group by gender
 0      m                 1      1                1
 3      m                 0      0                1
 4      m                 0      1                1]

In [8]:
# computing gini impurity for gender
gini_female_sample = gini_impurity(list(gender_groups[0].click))
gini_male_sample = gini_impurity(list(gender_groups[1].click))
weighted_sum = len(gender_groups[0])/len(sample_df)*gini_female_sample + \
               len(gender_groups[1])/len(sample_df)*gini_male_sample
    
print('gini impurity for split by gender')
print(f'gini_female_sample: {gini_female_sample} \ngini_male_sample: {gini_male_sample} \nweighted_sum: {weighted_sum}')

gini impurity for split by gender
gini_female_sample: 0.5 
gini_male_sample: 0.4444444444444444 
weighted_sum: 0.4666666666666667


In [9]:
# computing gini impurity for interest in tech
sample_tech = sample_df.groupby('interest in tech')
tech_groups = [group for key, group in sample_tech]
tech_groups

[  gender  interest in tech  click  group by gender
 1      f                 0      0                2
 3      m                 0      0                1
 4      m                 0      1                1,
   gender  interest in tech  click  group by gender
 0      m                 1      1                1
 2      f                 1      1                2]

In [10]:
# computing gini impurity for interest in tech
gini_nointerest_sample = gini_impurity(list(tech_groups[0].click))
gini_interest_sample = gini_impurity(list(tech_groups[1].click))
weighted_sum = len(tech_groups[0])/len(sample_df)*gini_nointerest_sample + \
               len(tech_groups[1])/len(sample_df)*gini_interest_sample

print('gini impurity for split by interest in tech')
print(f'gini_female_sample: {gini_female_sample} \ngini_male_sample: {gini_male_sample} \nweighted_sum: {weighted_sum}')

gini impurity for split by interest in tech
gini_female_sample: 0.5 
gini_male_sample: 0.4444444444444444 
weighted_sum: 0.26666666666666666


In [11]:
# we observe that a split by interest in tech has a lower gini impurity than splitting by gender
# therefore we observe a greater information gain if we were to split by 'interest in tech' than 'gender'

# lets perform the same computations, this time using entropy as the metric
def entropy(y):
    """
    objective: computes the entropy or probabilistic measure of uncertainty
    :param y: [int] - list of class labels
    :return: float
    """
    if not y:
        return 0
    counts = np.unique(y, return_counts=True)[1]
    fractions = counts / float(len(y))
    return - np.sum(fractions * np.log2(fractions))


print(entropy([1, 1, 0, 1, 0]))
print(entropy([1, 1, 0, 1, 0, 0]))
print(entropy([1, 1, 1, 1]))

0.9709505944546686
1.0
-0.0


In [12]:
# computing entropy for gender
en_female_sample = entropy(list(gender_groups[0].click))
en_male_sample = entropy(list(gender_groups[1].click))
weighted_sum = len(gender_groups[0])/len(sample_df)*en_female_sample + \
               len(gender_groups[1])/len(sample_df)*en_male_sample
    
print('entropy for split by gender')
print(f'en_female_sample: {en_female_sample} \nen_male_sample: {en_male_sample} \nweighted_sum: {weighted_sum}')

entropy for split by gender
en_female_sample: 1.0 
en_male_sample: 0.9182958340544896 
weighted_sum: 0.9509775004326937


In [13]:
# computing gini impurity for interest in tech
en_nointerest_sample = entropy(list(tech_groups[0].click))
en_interest_sample = entropy(list(tech_groups[1].click))
weighted_sum = len(tech_groups[0])/len(sample_df)*en_nointerest_sample + \
               len(tech_groups[1])/len(sample_df)*en_interest_sample

print('entropy for split by interest in tech')
print(f'en_nointerest_sample: {en_nointerest_sample}\nen_interest_sample: {en_interest_sample}\nweighted_sum: {weighted_sum}')

entropy for split by interest in tech
en_nointerest_sample: 0.9182958340544896
en_interest_sample: -0.0
weighted_sum: 0.5509775004326937


In [14]:
# combining the two impurities into a single function

criterion_function = {'gini': gini_impurity, 'entropy': entropy}
def weighted_impurity(groups, criterion='gini'):
    """
    objective: compute the weighted impurity of children after split
    :param groups: [[int]] - list of labels per group
    :criterion: str - metric that measures the quality of a split
    :return: float - weighted impurity
    """
    total_rows = sum(len(group) for group in groups)
    weights = np.array([len(group)/total_rows for group in groups])
    impurity = np.array([criterion_function[criterion](group) for group in groups])
    return np.sum(np.multiply(weights, impurity))


# sanity check
gini_gender = weighted_impurity([list(df.click) for key, df in sample_df.groupby('gender')])
gini_tech = weighted_impurity([list(df.click) for key, df in sample_df.groupby('interest in tech')])
en_gender = weighted_impurity([list(df.click) for key, df in sample_df.groupby('gender')], 'entropy')
en_tech = weighted_impurity([list(df.click) for key, df in sample_df.groupby('interest in tech')], 'entropy')

print(f'gini metric | split by gender: {gini_gender}')
print(f'gini metric | split by tech: {gini_tech}')
print(f'entropy metric | split by gender: {en_gender}')
print(f'entropy metric | split by tech: {en_tech}')


gini metric | split by gender: 0.4666666666666667
gini metric | split by tech: 0.26666666666666666
entropy metric | split by gender: 0.9509775004326937
entropy metric | split by tech: 0.5509775004326937


In [15]:
# now lets practice the algorithm, this time automatically selecting the correct attribut
# to split on

sample_df = pd.DataFrame(columns = ['user interest', 'user occupation', 'click'], 
                         data = [['tech', 'professional', 1],
                                 ['fashion', 'student', 0],
                                 ['fashion', 'professional', 0],
                                 ['sports', 'student', 0],
                                 ['tech', 'student', 1],
                                 ['tech', 'retired', 0],
                                 ['sports', 'professional', 1]])
sample_df

,user interest,user occupation,click
0,tech,professional,1
1,fashion,student,0
2,fashion,professional,0
3,sports,student,0
4,tech,student,1
5,tech,retired,0
6,sports,professional,1


In [1]:
# combining everything together
import pandas as pd
import numpy as np
import sys
import pprint


def gini_impurity(y):
    """
    objective: compute the gini impurity of a dataset
    :param y: [int] - the numerical classification or output
    :return: float
    """
    # when the set is empty, it is also pure
    if not y:
        return 0
    # count the occurrences of each label
    counts = np.unique(y, return_counts=True)[1]
    fractions = counts / float(len(y))
    return 1 - np.sum(fractions ** 2)


def entropy(y):
    """
    objective: computes the entropy or probabilistic measure of uncertainty
    :param y: [int] - list of class labels
    :return: float
    """
    if not y:
        return 0
    counts = np.unique(y, return_counts=True)[1]
    fractions = counts / float(len(y))
    return - np.sum(fractions * np.log2(fractions))


criterion_function = {'gini': gini_impurity, 'entropy': entropy}
def weighted_impurity(groups, criterion='gini'):
    """
    objective: compute the weighted impurity of children after split
    :param groups: [[int]] - list of labels per group
    :criterion: str - metric that measures the quality of a split
    :return: float - weighted impurity
    """
    total_rows = sum(len(group) for group in groups)
    weights = np.array([len(group)/total_rows for group in groups])
    impurity = np.array([criterion_function[criterion](group) for group in groups])
    return np.sum(np.multiply(weights, impurity))


class Node:
    def __init__(self, data, feature, impurity, left=None, right=None, leaf_class=None):
        """
        :param data: pd.DataFrame - subset data by a particular feature group
        :param feature: str - feature data is grouped by
        :param impurity: float- impurity metric that was made for the decision of the split
        :param left: Node - pointer to left child
        :param right: Node - pointer to right child
        :param leaf_class: int - dominant class represented by the leaf node
        """
        self.data = data
        self.feature = feature
        self.impurity = impurity
        self.left, self.right = left, right
        self.leaf_class = leaf_class

    def __repr__(self):
        shape = self.data.shape if self.data is not None else 'none'
        feature = self.feature if self.feature else 'none'
        impurity = self.impurity if self.impurity else 'none'
        return str({'data_shape': shape, 'feature': feature, 'impurity': impurity})


def get_dominate_class(data, y_label):
    """
    objective: find the most frequent class in ``data``
    :param data: pd.DataFrame - subset data by a particular feature group
    :param y_label: np.array - output data
    """
    if data.empty:
        return -1
    return np.bincount(np.array(data[y_label])).argmax()


def __split_node(data, by):
    """
    objective: split data ``by`` some attribute and apply one-vs-rest
    :param data: pd.DataFrame - subset data
    :param by: str - column to group by
    :return: [(pd.DataFrame, pd.DataFrame)] - (one, rest) dataframes. this can
    also be interpreted as (left, right) where left = unique, right = remainder
    """
    groups = [df for _, df in data.groupby(by)]
    one_rest = []
    for i in range(len(groups)):
        try:
            one, rest = groups[i], pd.concat([group for j, group in enumerate(groups) if j != i])
        except ValueError:
            # there was an empty concatenation
            one, rest = groups[i], pd.DataFrame(columns=list(data))
        one_rest.append((one, rest))
    return one_rest


def _get_best_split(data, criterion, y_label):
    """
    objective: identify the best attribute to split on ``data``
    :param data: pd.DataFrame - subset data
    :param criterion: str - 'gini' or 'entropy' decision metric
    :param y_label: str - label for output in data
    :return: dict - the score, left, right, and column for optimal split
    """

    def get_best_score(one_rest):
        # note: top score is the minimum score
        top_score, top_i = sys.maxsize, None
        for i, (one, rest) in enumerate(one_rest):
            labels = [list(one[y_label]), list(rest[y_label])]
            impurity = weighted_impurity(labels, criterion)
            if impurity < top_score:
                top_score, top_i = impurity, i
        return top_score, top_i

    # iterate through all the columns except the y_label (just X)
    # g is short for 'global'
    columns = list(filter(lambda col_name: col_name != y_label, list(data)))
    g_top_score, g_top_one_rest, g_top_col = sys.maxsize, None, None

    # iterate through every attribute in the data
    for col_name in columns:
        # get all the possible binary splits for a particular attribute
        one_rest = __split_node(data, col_name)
        top_score, top_i = get_best_score(one_rest)
        if g_top_score > top_score:
            g_top_score, g_top_one_rest, g_top_col = top_score, one_rest[top_i], col_name
    return {'impurity': g_top_score, 'left': g_top_one_rest[0],
            'right': g_top_one_rest[1], 'col_name': g_top_col}


def build_and_train_tree(data, max_depth, min_size, criterion='gini', y_label='y'):
    """
    objective: build the decision tree
    :param data: pd.DataFrame - training data from its entry point
    :max_depth: int - maximum depth of tree before having to stop
    :min_size: int - minimum number of samples in dataset as a result from the split
    before having to stop
    :criterion: str - impurity metric for finding optimal split
    :param y_label: str - label for output in data
    :return: Node - root of the decision tree
    """

    def dfs(data, depth):
        # checking terminating conditions (sufficient data, and depth)
        if len(data) == 0 or len(data) <= min_size or depth >= max_depth:
            return Node(None, None, None, leaf_class = get_dominate_class(data, y_label))
        # otherwise we are safe to split again
        else:
            # build the new node, and recurse to the next level.
            # connect the node back when on returns
            node_attributes = _get_best_split(data, criterion, y_label)
            cur_node = Node(data, node_attributes['col_name'], node_attributes['impurity'])
            cur_node.left = dfs(node_attributes['left'], depth + 1)
            cur_node.right = dfs(node_attributes['right'], depth + 1)
            return cur_node
    return dfs(data, 1)


def visualize_tree(root):
    def dfs(node, tab_count):
        delimit = '\t' * tab_count
        if node.leaf_class is None:
            print(delimit + str(node))
            dfs(node.left, tab_count + 1)
            dfs(node.right, tab_count + 1)
        else:
            print(f'{delimit}dominant class: {node.leaf_class}')
    dfs(root, 0)


def catagorical_test():
    # categorical test
    sample_df = pd.DataFrame(columns=['user interest', 'user occupation', 'click'],
                             data=[['tech', 'professional', 1],
                                   ['fashion', 'student', 0],
                                   ['fashion', 'professional', 0],
                                   ['sports', 'student', 0],
                                   ['tech', 'student', 1],
                                   ['tech', 'retired', 0],
                                   ['sports', 'professional', 1]])

    # no need to split data, but if we want to here's how we would do it
    # train_ratio = .66
    # mask = np.random.rand(len(sample_df)) < train_ratio
    # train, test = sample_df[mask], sample_df[~mask]
    root = build_and_train_tree(sample_df, 3, 2, y_label='click')
    visualize_tree(root)


def numerical_test():
    sample_df = pd.DataFrame(columns=['x1', 'x2', 'y'], data=[[6, 7, 0],
                                                              [2, 4, 0],
                                                              [7, 2, 0],
                                                              [3, 6, 0],
                                                              [4, 7, 0],
                                                              [5, 2, 1],
                                                              [1, 6, 1],
                                                              [2, 0, 1],
                                                              [6, 3, 1],
                                                              [4, 1, 1]])
    root = build_and_train_tree(sample_df, 4, 2, y_label='y')
    visualize_tree(root)


if __name__ == "__main__":
    catagorical_test()
    numerical_test()


{'data_shape': (7, 3), 'feature': 'user interest', 'impurity': 0.34285714285714286}
	dominant class: 0
	{'data_shape': (5, 3), 'feature': 'user occupation', 'impurity': 0.26666666666666666}
		dominant class: 1
		dominant class: 0
{'data_shape': (10, 3), 'feature': 'x2', 'impurity': 0.375}
	dominant class: 0
	{'data_shape': (8, 3), 'feature': 'x1', 'impurity': 0.35714285714285715}
		dominant class: 0
		{'data_shape': (7, 3), 'feature': 'x1', 'impurity': 0.238095238095238}
			dominant class: 0
			dominant class: 1


## Examples from Sklearn

In [8]:
from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier


iris = load_iris()
X, y = iris.data, iris.target
tree_clf = DecisionTreeClassifier()
tree_clf.fit(X, y)

# two prediction methods are support, one from the probabilities 
# for each class, and another for the classification itself
tree_clf.predict_proba(X)
tree_clf.predict(X)

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])